In [1]:
import processor
from data import Data, preprocess_midi_files_under
import torch
from model import load_config, load_model
import random

In [10]:
for path in ['2002', '2004', '2006', '2008', '2009', '2011', '2013', '2014', '2015', '2017', '2018']:
    preprocess_midi_files_under(f'C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/{path}', 'preprocess')

 [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\chan01.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\chan02.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\cho01.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\cho02.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\cho03.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\cho04.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\cho05.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\dossin01.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\dossin02.mid] [C:/Users/Draco/Documents/GitHub/MusicTransfor

In [2]:
# Example usage:
config = load_config()
batch_size = config["batch_size"]
block_size = config["block_size"]
max_iters = config["max_iters"]
eval_interval = config["eval_interval"]
save_interval = config["save_interval"]
learning_rate = config["learning_rate"]
eval_iters = config["eval_iters"]
n_embd = config["n_embd"]
n_head = config["n_head"]
n_layer = config["n_layer"]
dropout = config["dropout"]
vocab_size = config["vocab_size"]
device = config["device"]
generated_event_length = config["generated_event_length"]

In [ ]:
dataset = Data('preprocess').all_data()

torch.manual_seed(1337)

m = load_model()

data = torch.tensor(Data('preprocess').all_data())
# data = torch.tensor(dataset.batch(1,2000)[2], dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

def shift_note_sequence(sequence, rand_ints, lower_bound, upper_bound):
    # Clone the sequence to avoid modifying the original tensor
    shifted_sequence = sequence.clone()
    
    # Process each row independently
    for i in range(sequence.size(0)):
        # Define mask for elements in the specified range for this row
        mask = (sequence[i] >= lower_bound) & \
               (sequence[i] < upper_bound)
        
        # Apply the row-specific shift, clamping within 0, 127]
        shifted_sequence[i, mask] = torch.clamp(sequence[i, mask] + rand_ints[i], min=lower_bound, max=upper_bound - 1)
    
    return shifted_sequence

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    # Added pitch shifting during training
    note_r_ints = [random.randint(-6, 6) for _ in range(batch_size)]
    note_lb = processor.START_IDX["notes"]
    note_ub = processor.START_IDX["notes"] + processor.RANGE_NOTES
    new_x = shift_note_sequence(x, note_r_ints, note_lb, note_ub)
    new_y = shift_note_sequence(y, note_r_ints, note_lb, note_ub)
    
    # Added velocity shifting during training
    vel_r_ints = [random.randint(-10, 10) for _ in range(batch_size)]
    vel_lb = processor.START_IDX["velocity"]
    vel_ub = processor.START_IDX["velocity"] + processor.RANGE_VEL    
    new_x = shift_note_sequence(new_x, vel_r_ints, vel_lb, vel_ub)
    new_y = shift_note_sequence(new_y, vel_r_ints, vel_lb, vel_ub)
    
    new_x, new_y = new_x.to(device), new_y.to(device)
    return new_x, new_y

@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split) 
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % config['save_interval'] == config['save_interval'] - 1:
        torch.save(m.state_dict(), f'models/train_{losses["train"]:.4f}-val_{losses["val"]:.4f}.pth')

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Draco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Draco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1075, in launch

0.908165 M parameters
step 0: train loss 6.1095, val loss 6.1117
step 100: train loss 3.3157, val loss 3.3062
step 200: train loss 3.1915, val loss 3.1931
step 300: train loss 3.1204, val loss 3.1570
step 400: train loss 3.0923, val loss 3.1048
step 500: train loss 3.0388, val loss 3.0699
step 600: train loss 3.0251, val loss 3.0344
step 700: train loss 2.9837, val loss 2.9986
step 800: train loss 2.9882, val loss 2.9786
step 900: train loss 2.9647, val loss 2.9538
step 1000: train loss 2.9303, val loss 2.9367
step 1100: train loss 2.9245, val loss 2.9196
step 1200: train loss 2.9012, val loss 2.9147
step 1300: train loss 2.8877, val loss 2.8769
step 1400: train loss 2.8709, val loss 2.8812
step 1500: train loss 2.8631, val loss 2.8667
step 1600: train loss 2.8494, val loss 2.8563


In [9]:
torch.save(m.state_dict(), f'models/train_{losses["train"]:.4f}-val_{losses["val"]:.4f}.pth')
# m = load_model('2000_epochs.pth')

In [11]:
# generate from the model
context = torch.tensor([[306, 59, 150]], device=device)
output_sequence = m.generate(context, max_new_tokens=200)[0].tolist()

In [12]:
processor.decode_midi(output_sequence, "thing.mid")